In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from agents.tec_an import *
from agents.data_agent import *
from agents.stock_agent import *
import numpy as np
from data_util import *
import pandas as pd
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from bitstamp import *
from process_live_trade import *
from model_winner_select import *
from eval_model import *
from cache_providers import *

base_dir = "data/"

In [2]:
currency = "btcusd"

full_data = base_dir + "backtest/"
full_data = base_dir + "4jul21/"
#full_data = base_dir + "btcusd/"
data_gen = DataGenerator(random = False, base_dir = full_data)
data_gen.rewind()
data_count = (data_gen.steps - 100)
#data_count = 200000
raw = data_gen.next()

minute = 15

step = 1

hot_load = True

In [3]:
path = "model/"
files = os.listdir(path)
models = []
for file in files:
    models.append(f"{path}{file}")

models = ['model/ber_30_20']
    
val_start = 1629507199#int(raw['timestamp'])

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online_path = f'data/online{file_prefix}_{minutes}'
        
    return load(online_path)  

def get_provider(minu, win):
    tec = TecAn(windows = win, windows_limit = 100)
    sourceDataGenerator = SourceDataGenerator(tec = tec)
    provider = get_online_data(
            minutes = minu, 
            source_data_generator=sourceDataGenerator, 
            load_from_disk = True, 
            file_prefix=win
            )
        
    return provider

In [5]:
model_list = load(models[-1])
trained_model = model_list[-1]

In [6]:
minutes = trained_model.model_detail.data_detail.minutes
windows = trained_model.model_detail.data_detail.windows
steps_ahead = trained_model.model_detail.data_detail.steps_ahead
provider = get_provider(minu = minutes, win = windows)     

In [7]:

cache = CacheProvider(currency_list=provider.val_keys, verbose = False)


In [8]:
from collections import Counter
valX, valY, times = provider.load_val_data("btcusd")
x, y, closed_prices = get_sequencial_data(valX, valY, 3)
print(Counter(y))
#6657

Counter({1: 769, 0: 672})


In [9]:
stop_losses = ([0.0, -0.001, -0.01, -0.1, -1, -100])

In [10]:
model = trained_model.model_detail.model

for stop_loss in stop_losses:
    print(f"---- {stop_loss} ----")
    online = provider
    val_keys = online.val_keys
    for key in val_keys:
        back, metric = eval_model(model = model,
                          hot_load_total = 100,
                          currency = key, 
                          cache = cache,
                          stop_loss = stop_loss,
                          provider = online, 
                          step = step, 
                          verbose = False)
        print(f"{key}")
        back.report()
        print()


---- 0.0 ----
btcusd
0.43871% -> 100.43870815490845
Positive: 157(0.5258496815286623) Negative: 158(-0.5197493670886075)

ethusd
-16.80247% -> 83.19752870373128
Positive: 234(0.6001051282051282) Negative: 237(-0.6634071729957806)

ltcusd
-16.18183% -> 83.81817144542596
Positive: 200(0.6419460000000001) Negative: 221(-0.6541705882352941)

ltcbtc
-1.31666% -> 98.68334201928351
Positive: 42(0.4737119047619048) Negative: 48(-0.44191874999999997)

---- -0.001 ----
btcusd
-2.79173% -> 97.208266843295
Positive: 127(0.5594661417322834) Negative: 140(-0.5274607142857143)

ethusd
-16.80247% -> 83.19752870373128
Positive: 234(0.6001051282051282) Negative: 237(-0.6634071729957806)

ltcusd
-16.18183% -> 83.81817144542596
Positive: 200(0.6419460000000001) Negative: 221(-0.6541705882352941)

ltcbtc
-1.31666% -> 98.68334201928351
Positive: 42(0.4737119047619048) Negative: 48(-0.44191874999999997)

---- -0.01 ----
btcusd
-6.09883% -> 93.9011693636496
Positive: 129(0.5391984496124032) Negative: 136(-0.5

In [11]:
back.loss_limit

AttributeError: 'BackTest' object has no attribute 'loss_limit'

In [ ]:
btcusd -> {'recall': 0.6559633027522935, 'precision': 0.32987312572087657, 'f1': 0.4389869531849578, 'accuracy': 0.49271339347675225, 'roc_auc': 0.5389269250079876}
-3.92733% -> 96.07266706598205
Positive: 133(0.38393082706766923) Negative: 25(-2.199628)

ethusd -> {'recall': 0.7270955165692008, 'precision': 0.3790650406504065, 'f1': 0.4983299933199733, 'accuracy': 0.47883414295628035, 'roc_auc': 0.5343451720777038}
-13.95102% -> 86.04898326366862
Positive: 119(0.5106663865546218) Negative: 33(-2.2642515151515155)

ltcusd -> {'recall': 0.6704980842911877, 'precision': 0.3808487486398259, 'f1': 0.48577376821651624, 'accuracy': 0.4857737682165163, 'roc_auc': 0.5256734164655068}
-28.68033% -> 71.31966840346664
Positive: 84(0.5380202380952381) Negative: 28(-2.6383607142857146)

ltcbtc -> {'recall': 0.43360433604336046, 'precision': 0.28622540250447226, 'f1': 0.3448275862068965, 'accuracy': 0.5780707841776545, 'roc_auc': 0.5307014217530236}
-0.12002% -> 99.8799799967114
Positive: 85(0.2832105882352941) Negative: 25(-0.9676920000000001)


In [ ]:
results = []
for trained_model in model_list:
    reference_profit = {}
    models_profit = {}

    models_profit_metric = {}

    models_score = {}

    step = trained_model.model_detail.data_detail.steps_ahead

    profits = []
    for train_set in provider.val_keys:
        trainX_raw, trainY_raw, times = provider.load_val_data(train_set)
        x, y, closed_prices = get_sequencial_data(trainX_raw, trainY_raw, step)
        reference = get_max_profit(x, y, closed_prices, step)

        key = train_set

        reference_profit[key] = reference.get_profit()

        back, score = eval_model(
                trained_model.model_detail.model, 
                train_set, 
                step = step,
                cache = cache,
                provider = provider,
                hot_load_total = 100,
                verbose = False
                )
        
        #for metric_key in score:
        #    print(f"{metric_key} -> {score[metric_key]}")
        print(back.current)
        if (back.current > 100):
            back.report()
            results.append((back, score))
        #back.on_down(back.buy_price, back.buy_price)
        break


In [ ]:
winner = get_best_model(
    minutes_list=[minute],
    result_paths=models,
    currency_list=[currency],
    timestamp = val_start,
    winner_path = None
)

In [ ]:
result = winner

model = result['model']
windows = result['window']
minutes = result['minutes']
step = result['step']
profit = result['profit']
print(f"Minutes={minutes} Window={windows} Step={step} | {profit}")
print(f"{model}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def get_charts(stock_data):
    fig = plt.figure()
    ax = plt.axes()
    x = stock_data
    plt.plot(x, linewidth=1)
    #plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
    plt.show()


In [ ]:

on_raw_data = lambda raw: print(raw)

data_gen.rewind()

first = data_gen.next()

In [ ]:
agent_reference, back_reference, stock_reference = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True,
                                verbose = False)
prices = []
agent_reference.on_closed_price = lambda price: prices.append(price)
#agent.ohlc['open'].plot()

In [ ]:
data_gen.rewind()


reference_list = []
#data_count
for i in tqdm(range(data_count)):
    raw = data_gen.next()
    price = raw[PRICE_KEY]
    amount = raw[AMOUNT_KEY]
    timestamp = raw[TIMESTAMP_KEY]
    timestamp_pd = pd.to_datetime(timestamp, unit='s')
    reference_list.append([timestamp_pd, price])


In [ ]:
DATE = 'Date'
df = pd.DataFrame(agent_reference.list, columns = [DATE, CLOSE])
df = df.set_index(pd.DatetimeIndex(df[DATE]))

In [ ]:
time = df[CLOSE].resample(agent_reference.resample)
ohlc = time.ohlc()
print("All data reference")
#ohlc.plot()
#ohlc

In [ ]:
cache = CacheProvider([currency],  verbose = True)


In [ ]:
online = cache.get_provider(minutes, windows, val_start = 1629507700, val_end = 1629538206)

In [ ]:
cache = CacheProvider([currency],  verbose = True)

online = cache.get_provider(minutes, windows, val_start = 1629507700, val_end = 1629538206)

back, metric = eval_model(
    model = model, 
    currency = currency, 
    step = step,
    verbose = True, 
    provider = online,
    hot_load_total = 100,
    cache = cache
    )

back.report()

In [ ]:
agent_raw, back_raw, stock_raw = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                verbose = True,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)
prices = []

In [ ]:
agent_raw.last_index

In [ ]:
agent_raw.save_history = True
stock_raw.verbose = True
raw = data_gen.next()
agent_raw.on_new_raw_data(raw)

In [ ]:
agent_raw.last_index

In [ ]:
data_gen.rewind()
agent_raw.save_history = True
stock_raw.verbose = True

init_raw_process(currency, minutes, int(first['timestamp']), agent_raw)

#data_count
for i in range(data_count):
    raw = data_gen.next()
    agent_raw.on_new_raw_data(raw)

In [ ]:
val_start = int(first['timestamp'])
val_end = int(raw['timestamp'])

start_string = pd.to_datetime(val_start, unit='s')
end_string = pd.to_datetime(val_end, unit='s')

print(f"{start_string}({val_start}) -> {end_string}({val_end})")

In [ ]:
back_raw.on_down(back_raw.buy_price, back_raw.buy_price)
back_raw.report()

In [ ]:
agent_raw.ohlc[-50:]["open"].plot()
#agent.ohlc[1:]

In [ ]:
x_list, price_list, time_list = online.load_val_data(currency)

In [ ]:
get_charts(price_list)
print(len(price_list))

In [ ]:
agent_array, back_array, stock_array = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True,
                                verbose = False)

In [ ]:
agent_array.save_history = True
stock_array.verbose = True
for idx in range(len(price_list)):
    price = price_list[idx]
    time = time_list[idx]
    order = [[f"{price}", f"{price}"]]
    amount = 0.0
    agent_array.process_data(price, amount, time, order, order)

In [ ]:
back_array.on_down(back_array.buy_price, back_array.buy_price)
back_array.report()

In [ ]:
def eval_step_(model, train_set, step, provider, verbose = False):

    valX, valY, times = provider.load_val_data(train_set)
    
    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    history = []
    
    preds = model.predict(np.array(x))
    
    #for idx in range(len(x)):
    #    print(f"{times[idx]} -> {closed_prices[idx]} {y[idx]} ")

    metrics = {}
    metrics["recall"] = recall_score(y, preds)
    metrics["precision"] = precision_score(y, preds)
    metrics["f1"] = f1_score(y, preds)
    metrics["accuracy"] = accuracy_score(y, preds)
    metrics["roc_auc"] = roc_auc_score(y, preds)
    
    back = BackTest(value = 100, 
                    verbose = True, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    
    back = backtest_model(model, valX, valY, back)
    
    for idx in range(len(times)):
        timestamp = times[idx]
        price = closed_prices[idx]
        y = model.predict([valX[idx]])
        is_up = False
        if (y[0] > 0.5):
            is_up = True
        history.append((timestamp, price, x, is_up))
    
    get_charts(valY)

    
    return back, metrics, history
    

In [ ]:
back_eval, metr, history = eval_step_(model, currency, step, online, verbose = False)
back_eval.on_down(back_eval.buy_price, back_eval.buy_price)
back_eval.report()

In [ ]:
for data in history:
    print(f"{data[0]} - {data[1]} - {data[3]}")

In [ ]:
print(metrics)


In [ ]:
back_raw.report()
#-1.78465% -> 98.21535030676218

In [ ]:
back_array.report()
#-1.89717% -> 98.10282664807247